In [1]:
import numpy as np
import pandas as pd
import scipy
import os
from sklearn.utils.extmath import stable_cumsum
import math
import matplotlib.pyplot as plt
import pickle
import statsmodels.distributions.empirical_distribution as edf
from scipy.interpolate import interp1d
from sklearn.metrics.pairwise import cosine_similarity
import functools
import scipy as sp
from sklearn.model_selection import StratifiedKFold, cross_validate
from scipy.spatial import distance
from sklearn.decomposition import PCA,qPCA
from tabulate import tabulate
import mpld3
from statsmodels.distributions.empirical_distribution import ECDF
from Models.PCAmodel import Model
mpld3.enable_notebook()

from sklearn.QuantumUtility.Utility import *
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('kddcup.data_10_percent_corrected',header = None)
df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.00,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,tcp,http,SF,310,1881,0,0,0,0,...,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal.
494017,0,tcp,http,SF,282,2286,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal.
494018,0,tcp,http,SF,203,1200,0,0,0,0,...,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal.
494019,0,tcp,http,SF,291,1200,0,0,0,0,...,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal.


In [3]:
df1 = df.drop(columns=[1,2,3,6,11,20,21])
df1.loc[df1[41] != 'normal.', 41] = 'attack'
df1

,0,4,5,7,8,9,10,12,13,14,...,32,33,34,35,36,37,38,39,40,41
0,0,181,5450,0,0,0,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.00,0.00,0.0,0.0,normal.
1,0,239,486,0,0,0,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal.
2,0,235,1337,0,0,0,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
3,0,219,1337,0,0,0,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
4,0,217,2032,0,0,0,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,310,1881,0,0,0,0,0,0,0,...,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal.
494017,0,282,2286,0,0,0,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal.
494018,0,203,1200,0,0,0,0,0,0,0,...,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal.
494019,0,291,1200,0,0,0,0,0,0,0,...,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal.


In [5]:
def systematic_sampling(df,pop_length):
    df_normal=df[df[41]!='attack']
    step = int(len(df_normal)/pop_length)
    
    
    first_index = random.choice(np.arange(step))
    indexes = np.arange(first_index, len(df), step=step)[:5000]
    
    training_set = df_normal.iloc[indexes]
    
    mean_std = training_set.drop(columns=41).describe(include='all').loc[['mean','std']]
    
    index_res = list(set(df_normal.index)-set(training_set.index))
    test = df_normal.loc[index_res]
    t = df[df[41] =='attack'].sample(39674)
    test= test.append(t)

    return training_set,test,mean_std


In [6]:
train1,test1,mean1 = systematic_sampling(df1,5000)

train2,test2,mean2 = systematic_sampling(df1,5000)

train3,test3,mean3 = systematic_sampling(df1,5000)

train4,test4,mean4 = systematic_sampling(df1,5000)

train5,test5,mean5 = systematic_sampling(df1,5000)

In [7]:
mean_std = [mean1,mean2,mean3,mean4,mean5]
test = [test1,test2,test3,test4,test5]

# ---------------------------------------------------------------------------------------------------------------

# Trimming function

In [8]:
def trimming(n_trimming, df, gamma):

    for n_tr in range(n_trimming):
        mean_array = np.array(df.drop(columns=41).describe(include='all').loc[['mean','std']].iloc[0])
        cov = np.cov(df.drop(columns=41).T)
        Icov = np.linalg.pinv(cov)
        dist_list = []
        for i in range(len(df)):
            sample_i = np.array(df.drop(columns=41).iloc[i])
            dist =distance.mahalanobis(sample_i,mean_array,Icov)**2
            dist_list.append(dist)
            #print(i)
        threshold = int(gamma*len(df))
        dist_list=np.array(dist_list)
        x = np.argpartition(dist_list,-threshold)[-threshold:]
        names =[]
    
        for i in x:
            names.append(df.iloc[i].name)
        df = df.drop(index = names)
    return df


## Trimming 1

Trimming the whole dataset and then extract 5000 samples from the training set.

In [10]:

#trimming 1 time
df1_normal = df1[df1[41]!='attack']
train = trimming(1,df1_normal,0.005)
train_trim=train.sample(5000)
#train_trim,test_trim,mean_trim = systematic_sampling(df1,5000)
mean_std_trim = train_trim.drop(columns=41).describe(include='all').loc[['mean','std']]
##TEST SET

x = list(train_trim.index)
index_res = list(set(df1_normal.index)-set(x))
test_trim = df1_normal.loc[index_res]
t = df1[df1[41] =='attack'].sample(39674)
test_trim= test_trim.append(t)
test_trim


NameError: name 'trimming' is not defined

## Trimming 2

Trimming the 5000 connections of the training set

In [12]:
train1 = trimming(1,train1,0.005)
train2 = trimming(1,train2,0.005)
train3 = trimming(1,train3,0.005)
train4 = trimming(1,train4,0.005)
train5 = trimming(1,train5,0.005)

In [14]:
trains = [train1,train2,train3,train4,train5]

# ---------------------------------------------------------------------------------------------------------------

## Normalize training set and test set


In [15]:
scaler = StandardScaler()
for train_ in trains:
    for (columnName, columnData) in train_.drop(columns=41).iteritems():
        train_[columnName] = scaler.fit_transform(train_.drop(columns=41)[[columnName]])
#Remove mean and std from test set (Normalize test set)
tests =[]
for en, test_ in enumerate(test):
    test_ = (test_.drop(columns=41).sub(mean_std[en].loc['mean'],axis = 'columns')).div(mean_std[en].loc['std'],axis = 'columns').fillna(0)
    tests.append(test_)


# Load Train/Test

In [4]:
cwd = os.getcwd()
trains =[]
tests = []
for filename in os.listdir(os.path.join(cwd,'KDDCUP/Trains/')):
    trains.append(pd.read_csv(os.path.join(cwd,'KDDCUP/Trains/',filename)).drop(columns = ['Unnamed: 0', '41']))
    
for filename in os.listdir(os.path.join(cwd,'KDDCUP/Tests/')):
    tests.append(pd.read_csv(os.path.join(cwd,'KDDCUP/Tests/',filename)).set_index('Unnamed: 0'))

In [5]:
test = []
labels = []
for i in range(5):
    x = tests[i].index
    test.append(df1.loc[x])
    labels.append(df1.loc[x][41])
labels[0]

Unnamed: 0
0         normal.
1         normal.
2         normal.
3         normal.
5         normal.
           ...   
195319     attack
446383     attack
356582     attack
121429     attack
468177     attack
Name: 41, Length: 131952, dtype: object

# Fit PCA 

In [11]:
#Fit PCAs
pca30 = qPCA(svd_solver="full", name='pca30').fit(trains[0], classic_ret_variance_components=0.30)

pca40 = qPCA(svd_solver="full", name='pca40').fit(trains[1], classic_ret_variance_components=0.40)

pca50 = qPCA(svd_solver="full", name='pca50').fit(trains[2], classic_ret_variance_components=0.50)

pca60 = qPCA(svd_solver="full", name='pca60').fit(trains[3], classic_ret_variance_components=0.60)

pca70 = qPCA(svd_solver="full", name='pca70').fit(trains[4], classic_ret_variance_components=0.70)

PCAs=[pca30,pca40,pca50,pca60,pca70]


In [10]:
#Fit PCAs
pca30 = qPCA(svd_solver="full").fit(trains[0].drop(columns=41),classic_ret_variance_components=0.30)

pca40 = qPCA(svd_solver="full").fit(trains[1].drop(columns=41),classic_ret_variance_components=0.40)

pca50 = qPCA(svd_solver="full").fit(trains[2].drop(columns=41),classic_ret_variance_components=0.50)

pca60 = qPCA(svd_solver="full").fit(trains[3].drop(columns=41),classic_ret_variance_components=0.60)

pca70 = qPCA(svd_solver="full").fit(trains[4].drop(columns=41),classic_ret_variance_components=0.70)

PCAs=[pca30,pca40,pca50,pca60,pca70]


# Empirical Distribution

In [6]:
# Compute Quantile---> 1-\alpha1
alpha = [0.01,0.02,0.04,0.06,0.08,0.10]
quantils = []
percentiles = []
for i in alpha:
    eq =  [1,-2,i]
    quantile = 1-np.round(np.roots(eq)[1],decimals=4)
    quantils.append(quantile)
    percentiles.append(quantile*100)
quantils
percentiles

[99.5, 98.99, 97.98, 96.95, 95.92, 94.87]

Found outlier threshold using empirical distribution of \sum_q(y_i**2)/\lambda_i and of \sum_p-r+1(y_i**2)/\lambda_i

In [8]:
model = Model(PCAs, quantils).fit(trains,minor_sv_variance=0.20)

#recall, precision, accuracy, f1_score = model.predict(tests,labels,name_negative_class = 'normal.',only_dot_product=False, experiment=0)

In [10]:
recall_dot, precision_dot, accuracy_dot, f1_score_dot = model.predict(tests,labels,name_negative_class = 'normal.',only_dot_product=True, experiment=0)

pca30
(22.197239064164407,)
detection_rate: 0.6713212683369462
precision: 0.9849123585533615
accuracy: 0.8980841518127805
F1_score: 0.7984291624198094
TP: 26634 TN: 91870 FP: 408 FN: 13040 TOT_SAMPLES: 131952 LEN_TEST: 131952
(19.99135862820585,)
detection_rate: 0.6881836971316228
precision: 0.9715332882610398
accuracy: 0.9001834000242512
F1_score: 0.8056715404930876
TP: 27303 TN: 91478 FP: 800 FN: 12371 TOT_SAMPLES: 131952 LEN_TEST: 131952
(18.197348601053637,)
detection_rate: 0.7117507687654383
precision: 0.9428065840873426
accuracy: 0.9003501273190251
F1_score: 0.8111454219030519
TP: 28238 TN: 90565 FP: 1713 FN: 11436 TOT_SAMPLES: 131952 LEN_TEST: 131952
(17.001445027751092,)
detection_rate: 0.7136159701567777
precision: 0.9121721760422707
accuracy: 0.8932339032375409
F1_score: 0.8007693177961307
TP: 28312 TN: 89552 FP: 2726 FN: 11362 TOT_SAMPLES: 131952 LEN_TEST: 131952
(16.0683386812283,)
detection_rate: 0.7138932298230579
precision: 0.8889830508474577
accuracy: 0.8871710925184916

In [11]:
recall_dot1, precision_dot1, accuracy_dot1, f1_score_dot1 = model.predict(tests,labels,name_negative_class = 'normal.',only_dot_product=True, experiment=1)

exp1
pca30
22.197239064164407 224.59356683131793
detection_rate: 0.9427584816252458
precision: 0.975866207472344
accuracy: 0.9757790711773978
F1_score: 0.9590266916232918
TP: 37403 TN: 91353 FP: 925 FN: 2271 TOT_SAMPLES: 131952 LEN_TEST: 131952
19.99135862820585 148.32209883024575
detection_rate: 0.9610828250239452
precision: 0.9572224732640457
accuracy: 0.9753849884806597
F1_score: 0.9591487649041607
TP: 38130 TN: 90574 FP: 1704 FN: 1544 TOT_SAMPLES: 131952 LEN_TEST: 131952
18.197348601053637 64.72320606526893
detection_rate: 0.9891868730150728
precision: 0.9160617165799118
accuracy: 0.9694964835697829
F1_score: 0.9512209901230078
TP: 39245 TN: 88682 FP: 3596 FN: 429 TOT_SAMPLES: 131952 LEN_TEST: 131952
17.001445027751092 47.4282310649588
detection_rate: 0.9901950899833645
precision: 0.8785052998792433
accuracy: 0.9558778949921184
F1_score: 0.9310124182386956
TP: 39285 TN: 86845 FP: 5433 FN: 389 TOT_SAMPLES: 131952 LEN_TEST: 131952
16.0683386812283 38.26045873662737
detection_rate: 0.

In [10]:
recall_dot_corr_cos, precision_dot_corr_cos, accuracy_dot_corr_cos, f1_score_dot_corr_cos = model.predict(tests,labels,name_negative_class = 'normal.',only_dot_product=False, experiment=1)

exp1
pca30
22.197239064164407 224.59356683131793 0.20883486809554963 6.785217443519271
detection_rate: 0.9432625901093915
precision: 0.950908397916402
accuracy: 0.9682990784527707
F1_score: 0.9470700628883068
TP: 37423 TN: 90346 FP: 1932 FN: 2251 TOT_SAMPLES: 131952 LEN_TEST: 131952
19.99135862820585 148.32209883024575 0.20771018596844643 4.855580925637226
detection_rate: 0.9616121389322982
precision: 0.917289798273665
accuracy: 0.9623878380016976
F1_score: 0.9389281978711622
TP: 38151 TN: 88838 FP: 3440 FN: 1523 TOT_SAMPLES: 131952 LEN_TEST: 131952
18.197348601053637 64.72320606526893 0.2047289501833882 3.336467696600948
detection_rate: 0.9892624892876947
precision: 0.8549831173074829
accuracy: 0.9463213895962168
F1_score: 0.9172343682445461
TP: 39248 TN: 85621 FP: 6657 FN: 426 TOT_SAMPLES: 131952 LEN_TEST: 131952
17.001445027751092 47.4282310649588 0.20150159556487127 2.5964117047476614
detection_rate: 0.9902202954075717
precision: 0.8054371002132196
accuracy: 0.9251394446465382
F1_s

In [8]:
model = Model(PCAs, quantils).fit(trains,minor_sv_variance=0.20)
recall_dot, precision_dot, accuracy_dot, f1_score_dot = model.predict(tests,labels,name_negative_class = 'normal.',only_dot_product=False, experiment=1)

exp1
pca30
22.197239064164407 224.59356683131793 0.20883486809554963 6.785217443519271 0.03381291317437779 119.43986200471953
detection_rate: 0.961334879266018
precision: 0.944059405940594
accuracy: 0.9712471201649084
F1_score: 0.9526188275844844
TP: 38140 TN: 90018 FP: 2260 FN: 1534 TOT_SAMPLES: 131952 LEN_TEST: 131952
19.99135862820585 148.32209883024575 0.20771018596844643 4.855580925637226 0.03021833706261407 94.35498902963452
detection_rate: 0.9861370166859909
precision: 0.8953884883853988
accuracy: 0.9611904328846853
F1_score: 0.9385742902037927
TP: 39124 TN: 87707 FP: 4571 FN: 550 TOT_SAMPLES: 131952 LEN_TEST: 131952
18.197348601053637 64.72320606526893 0.2047289501833882 3.3364676966009488 0.021141061837545733 65.26404402947841
detection_rate: 0.9898170086202551
precision: 0.8156949089171842
accuracy: 0.929694131199224
F1_score: 0.8943598619857203
TP: 39270 TN: 83405 FP: 8873 FN: 404 TOT_SAMPLES: 131952 LEN_TEST: 131952
17.001445027751092 47.4282310649588 0.20150159556487135 2.

In [16]:
components = [i for i in recall_dot]
headers = ['FAR','PCA30','PCA40','PCA50','PCA60','PCA70','PCA90']
    
one_perc = [recall_dot[i][0] for i in components]
two_perc = [recall_dot[i][1] for i in components]
four_perc = [recall_dot[i][2] for i in components]
six_perc = [recall_dot[i][3] for i in components]
ten_perc = [recall_dot[i][4] for i in components]
thirty_perc = [recall_dot[i][5] for i in components]



one_perc_prec = [precision_dot[i][0] for i in components]
two_perc_prec = [precision_dot[i][1] for i in components]
four_perc_prec = [precision_dot[i][2] for i in components]
six_perc_prec = [precision_dot[i][3] for i in components]
ten_perc_prec = [precision_dot[i][4] for i in components]
thir_perc_prec = [precision_dot[i][5] for i in components]

one_perc_prec.insert(0,'1%')
two_perc_prec.insert(0,'2%')
four_perc_prec.insert(0,'4%')
six_perc_prec.insert(0,'6%')
ten_perc_prec.insert(0,'8%')
thir_perc_prec.insert(0,'10%')


one_perc.insert(0,'1%')
two_perc.insert(0,'2%')
four_perc.insert(0,'4%')
six_perc.insert(0,'6%')
ten_perc.insert(0,'8%')
thirty_perc.insert(0,'10%')


######
one_perc_f1 = [f1_score_dot[i][0] for i in components]
two_perc_f1 = [f1_score_dot[i][1] for i in components]
four_perc_f1 = [f1_score_dot[i][2] for i in components]
six_perc_f1 = [f1_score_dot[i][3] for i in components]
ten_perc_f1 = [f1_score_dot[i][4] for i in components]
thir_perc_f1 = [f1_score_dot[i][5] for i in components]

one_perc_f1.insert(0,'1%')
two_perc_f1.insert(0,'2%')
four_perc_f1.insert(0,'4%')
six_perc_f1.insert(0,'6%')
ten_perc_f1.insert(0,'8%')
thir_perc_f1.insert(0,'10%')


one_perc_acc = [accuracy_dot[i][0] for i in components]
two_perc_acc = [accuracy_dot[i][1] for i in components]
four_perc_acc = [accuracy_dot[i][2] for i in components]
six_perc_acc = [accuracy_dot[i][3] for i in components]
ten_perc_acc = [accuracy_dot[i][4] for i in components]
thir_perc_acc = [accuracy_dot[i][5] for i in components]



one_perc_acc.insert(0,'1%')
two_perc_acc.insert(0,'2%')
four_perc_acc.insert(0,'4%')
six_perc_acc.insert(0,'6%')
ten_perc_acc.insert(0,'8%')
thir_perc_acc.insert(0,'10%')



print('Detection Rate')
print(tabulate([one_perc, two_perc, four_perc, six_perc, ten_perc, thirty_perc], headers=headers))
print("\n \n ")
print('Precision')
print(tabulate([one_perc_prec, two_perc_prec, four_perc_prec, six_perc_prec, ten_perc_prec, thir_perc_prec], headers=headers))
print("\n \n ")
print('F1_Score')
print(tabulate([one_perc_f1, two_perc_f1, four_perc_f1, six_perc_f1, ten_perc_f1, thir_perc_f1], headers=headers))
print("\n \n ")
print('Accuracy')
print(tabulate([one_perc_acc, two_perc_acc, four_perc_acc, six_perc_acc, ten_perc_acc, thir_perc_acc], headers=headers))


Detection Rate
FAR       PCA30     PCA40     PCA50     PCA60     PCA70
-----  --------  --------  --------  --------  --------
1%     0.671321  0.957756  0.932878  0.935474  0.931492
2%     0.688184  0.98533   0.972123  0.961889  0.931945
4%     0.711751  0.985734  0.98465   0.986137  0.960478
6%     0.713616  0.986818  0.98591   0.987347  0.985104
8%     0.713893  0.987296  0.987221  0.988053  0.986767
10%    0.714196  0.987473  0.987448  0.989036  0.994404

 
 
Precision
FAR       PCA30     PCA40     PCA50     PCA60     PCA70
-----  --------  --------  --------  --------  --------
1%     0.984912  0.984608  0.993024  0.990129  0.986335
2%     0.971533  0.974353  0.98105   0.98224   0.981863
4%     0.942807  0.957333  0.958744  0.959533  0.965173
6%     0.912172  0.935821  0.932998  0.94003   0.942009
8%     0.888983  0.919915  0.911496  0.928998  0.920157
10%    0.864927  0.896971  0.89447   0.916606  0.900566

 
 
F1_Score
FAR       PCA30     PCA40     PCA50     PCA60     PCA70
----

In [13]:

components = [i for i in recall_dot1]
headers = ['FAR','PCA30','PCA40','PCA50','PCA60','PCA70','PCA90']
    
one_perc = [recall_dot1[i][0] for i in components]
two_perc = [recall_dot1[i][1] for i in components]
four_perc = [recall_dot1[i][2] for i in components]
six_perc = [recall_dot1[i][3] for i in components]
ten_perc = [recall_dot1[i][4] for i in components]
thirty_perc = [recall_dot1[i][5] for i in components]



one_perc_prec = [precision_dot1[i][0] for i in components]
two_perc_prec = [precision_dot1[i][1] for i in components]
four_perc_prec = [precision_dot1[i][2] for i in components]
six_perc_prec = [precision_dot1[i][3] for i in components]
ten_perc_prec = [precision_dot1[i][4] for i in components]
thir_perc_prec = [precision_dot1[i][5] for i in components]

one_perc_prec.insert(0,'1%')
two_perc_prec.insert(0,'5%')
four_perc_prec.insert(0,'7%')
six_perc_prec.insert(0,'9%')
ten_perc_prec.insert(0,'10%')
thir_perc_prec.insert(0,'20%')


one_perc.insert(0,'1%')
two_perc.insert(0,'5%')
four_perc.insert(0,'7%')
six_perc.insert(0,'9%')
ten_perc.insert(0,'10%')
thirty_perc.insert(0,'20%')


######
one_perc_f1 = [f1_score_dot1[i][0] for i in components]
two_perc_f1 = [f1_score_dot1[i][1] for i in components]
four_perc_f1 = [f1_score_dot1[i][2] for i in components]
six_perc_f1 = [f1_score_dot1[i][3] for i in components]
ten_perc_f1 = [f1_score_dot1[i][4] for i in components]
thir_perc_f1 = [f1_score_dot1[i][5] for i in components]

one_perc_f1.insert(0,'1%')
two_perc_f1.insert(0,'5%')
four_perc_f1.insert(0,'7%')
six_perc_f1.insert(0,'9%')
ten_perc_f1.insert(0,'10%')
thir_perc_f1.insert(0,'20%')


one_perc_acc = [accuracy_dot1[i][0] for i in components]
two_perc_acc = [accuracy_dot1[i][1] for i in components]
four_perc_acc = [accuracy_dot1[i][2] for i in components]
six_perc_acc = [accuracy_dot1[i][3] for i in components]
ten_perc_acc = [accuracy_dot1[i][4] for i in components]
thir_perc_acc = [accuracy_dot1[i][5] for i in components]



one_perc_acc.insert(0,'1%')
two_perc_acc.insert(0,'5%')
four_perc_acc.insert(0,'7%')
six_perc_acc.insert(0,'9%')
ten_perc_acc.insert(0,'10%')
thir_perc_acc.insert(0,'20%')



print('Detection Rate')
print(tabulate([one_perc, two_perc, four_perc, six_perc, ten_perc, thirty_perc], headers=headers))
print("\n \n ")
print('Precision')
print(tabulate([one_perc_prec, two_perc_prec, four_perc_prec, six_perc_prec, ten_perc_prec, thir_perc_prec], headers=headers))
print("\n \n ")
print('F1_Score')
print(tabulate([one_perc_f1, two_perc_f1, four_perc_f1, six_perc_f1, ten_perc_f1, thir_perc_f1], headers=headers))
print("\n \n ")
print('Accuracy')
print(tabulate([one_perc_acc, two_perc_acc, four_perc_acc, six_perc_acc, ten_perc_acc, thir_perc_acc], headers=headers))


Detection Rate
FAR       PCA30     PCA40     PCA50     PCA60     PCA70
-----  --------  --------  --------  --------  --------
1%     0.942758  0.985759  0.985356  0.986465  0.986238
5%     0.961083  0.987044  0.98654   0.987599  0.986692
7%     0.989187  0.988002  0.987322  0.98891   0.988002
9%     0.990195  0.989439  0.987775  0.989565  0.988406
10%    0.990724  0.990372  0.989792  0.989893  0.989338
20%    0.991556  0.991758  0.990976  0.991909  0.995967

 
 
Precision
FAR       PCA30     PCA40     PCA50     PCA60     PCA70
-----  --------  --------  --------  --------  --------
1%     0.975866  0.97129   0.978646  0.976472  0.972656
5%     0.957222  0.950947  0.958022  0.957363  0.959649
7%     0.916062  0.918782  0.912078  0.920791  0.916119
9%     0.878505  0.88748   0.88705   0.891665  0.888884
10%    0.848355  0.859913  0.85802   0.878807  0.861164
20%    0.818386  0.831386  0.831486  0.8577    0.832575

 
 
F1_Score
FAR       PCA30     PCA40     PCA50     PCA60     PCA70
----

In [14]:
components = [i for i in recall_dot1]
headers = ['FAR','PCC']
one_perc = [recall_dot1[i][0] for i in components]
two_perc = [recall_dot1[i][1] for i in components]
four_perc = [recall_dot1[i][2] for i in components]
six_perc = [recall_dot1[i][3] for i in components]
eight_perc = [recall_dot1[i][4] for i in components]
ten_perc = [recall_dot1[i][5] for i in components]

one_perc_avg = np.mean(one_perc)
one_perc_std = np.std(one_perc)

two_perc_avg = np.mean(two_perc)
two_perc_std = np.std(two_perc)

four_perc_avg = np.mean(four_perc)
four_perc_std = np.std(four_perc)

six_perc_avg = np.mean(six_perc)
six_perc_std = np.std(six_perc)


eight_perc_avg = np.mean(eight_perc)
eight_perc_std = np.std(eight_perc)

ten_perc_avg = np.mean(ten_perc)
ten_perc_std = np.std(ten_perc)

one_str = [str(one_perc_avg)+' +/- '+str(one_perc_std)]
two_str = [str(two_perc_avg)+' +/- '+str(two_perc_std)]
four_str = [str(four_perc_avg)+' +/- '+str(four_perc_std)]
six_str = [str(six_perc_avg)+' +/- '+str(six_perc_std)]
eight_str =[ str(eight_perc_avg)+' +/- '+str(eight_perc_std)]
ten_str = [str(ten_perc_avg)+' +/- '+str(ten_perc_std)]

one_str.insert(0,'1%')
two_str.insert(0,'2%')
four_str.insert(0,'4%')
six_str.insert(0,'6%')
eight_str.insert(0,'8%')
ten_str.insert(0,'10%')

print(tabulate([one_str, two_str, four_str, six_str, eight_str, ten_str], headers=headers))


FAR    PCC
-----  ---------------------------------------------
1%     0.9773151182134395 +/- 0.017282585775286387
2%     0.9817916015526542 +/- 0.010360791946994175
4%     0.9882845188284518 +/- 0.0006769709602744052
6%     0.9890759691485609 +/- 0.0008676818557268922
8%     0.9900236930987549 +/- 0.00048020136932439055
10%    0.9924333316529717 +/- 0.0017950368067565147


# DOT COSINE CORRELATION

In [14]:
components = [i for i in recall]
headers = ['FAR','PCA30','PCA40','PCA50','PCA60','PCA70','PCA90']
    
one_perc = [recall_dot1[i][0] for i in components]
two_perc = [recall_dot1[i][1] for i in components]
four_perc = [recall_dot1[i][2] for i in components]
six_perc = [recall_dot1[i][3] for i in components]
ten_perc = [recall_dot1[i][4] for i in components]
thirty_perc = [recall_dot1[i][5] for i in components]



one_perc_prec = [precision_dot1[i][0] for i in components]
two_perc_prec = [precision_dot1[i][1] for i in components]
four_perc_prec = [precision_dot1[i][2] for i in components]
six_perc_prec = [precision_dot1[i][3] for i in components]
ten_perc_prec = [precision_dot1[i][4] for i in components]
thir_perc_prec = [precision_dot1[i][5] for i in components]

one_perc_prec.insert(0,'1%')
two_perc_prec.insert(0,'5%')
four_perc_prec.insert(0,'7%')
six_perc_prec.insert(0,'9%')
ten_perc_prec.insert(0,'10%')
thir_perc_prec.insert(0,'20%')


one_perc.insert(0,'1%')
two_perc.insert(0,'5%')
four_perc.insert(0,'7%')
six_perc.insert(0,'9%')
ten_perc.insert(0,'10%')
thirty_perc.insert(0,'20%')


######
one_perc_f1 = [f1_score_dot1[i][0] for i in components]
two_perc_f1 = [f1_score_dot1[i][1] for i in components]
four_perc_f1 = [f1_score_dot1[i][2] for i in components]
six_perc_f1 = [f1_score_dot1[i][3] for i in components]
ten_perc_f1 = [f1_score_dot1[i][4] for i in components]
thir_perc_f1 = [f1_score_dot1[i][5] for i in components]

one_perc_f1.insert(0,'1%')
two_perc_f1.insert(0,'5%')
four_perc_f1.insert(0,'7%')
six_perc_f1.insert(0,'9%')
ten_perc_f1.insert(0,'10%')
thir_perc_f1.insert(0,'20%')


one_perc_acc = [accuracy_dot1[i][0] for i in components]
two_perc_acc = [accuracy_dot1[i][1] for i in components]
four_perc_acc = [accuracy_dot1[i][2] for i in components]
six_perc_acc = [accuracy_dot1[i][3] for i in components]
ten_perc_acc = [accuracy_dot1[i][4] for i in components]
thir_perc_acc = [accuracy_dot1[i][5] for i in components]



one_perc_acc.insert(0,'1%')
two_perc_acc.insert(0,'5%')
four_perc_acc.insert(0,'7%')
six_perc_acc.insert(0,'9%')
ten_perc_acc.insert(0,'10%')
thir_perc_acc.insert(0,'20%')



print('Detection Rate')
print(tabulate([one_perc, two_perc, four_perc, six_perc, ten_perc, thirty_perc], headers=headers))
print("\n \n ")
print('Precision')
print(tabulate([one_perc_prec, two_perc_prec, four_perc_prec, six_perc_prec, ten_perc_prec, thir_perc_prec], headers=headers))
print("\n \n ")
print('F1_Score')
print(tabulate([one_perc_f1, two_perc_f1, four_perc_f1, six_perc_f1, ten_perc_f1, thir_perc_f1], headers=headers))
print("\n \n ")
print('Accuracy')
print(tabulate([one_perc_acc, two_perc_acc, four_perc_acc, six_perc_acc, ten_perc_acc, thir_perc_acc], headers=headers))


Detection Rate
FAR       PCA30     PCA40     PCA50     PCA60     PCA70
-----  --------  --------  --------  --------  --------
1%     0.942758  0.985759  0.985356  0.986412  0.986238
5%     0.961083  0.987044  0.98654   0.987551  0.986692
7%     0.989187  0.988002  0.987322  0.988867  0.988002
9%     0.990195  0.989439  0.987775  0.989525  0.988406
10%    0.990724  0.990372  0.989792  0.989853  0.989338
20%    0.991556  0.991758  0.990976  0.991878  0.995967

 
 
Precision
FAR       PCA30     PCA40     PCA50     PCA60     PCA70
-----  --------  --------  --------  --------  --------
1%     0.975866  0.971314  0.97872   0.976626  0.972728
5%     0.957222  0.95097   0.958093  0.957438  0.95972
7%     0.916062  0.918804  0.912141  0.920723  0.916184
9%     0.878505  0.8875    0.887111  0.89149   0.888944
10%    0.848355  0.859932  0.858076  0.878588  0.861221
20%    0.818386  0.831403  0.831539  0.857412  0.832627

 
 
F1_Score
FAR       PCA30     PCA40     PCA50     PCA60     PCA70
-----

In [15]:
components = [i for i in recall]
headers = ['FAR','PCA30','PCA40','PCA50','PCA60','PCA70','PCA90']
    
one_perc = [recall_dot_corr_cos[i][0] for i in components]
two_perc = [recall_dot_corr_cos[i][1] for i in components]
four_perc = [recall_dot_corr_cos[i][2] for i in components]
six_perc = [recall_dot_corr_cos[i][3] for i in components]
ten_perc = [recall_dot_corr_cos[i][4] for i in components]
thirty_perc = [recall_dot_corr_cos[i][5] for i in components]



one_perc_prec = [precision_dot_corr_cos[i][0] for i in components]
two_perc_prec = [precision_dot_corr_cos[i][1] for i in components]
four_perc_prec = [precision_dot_corr_cos[i][2] for i in components]
six_perc_prec = [precision_dot_corr_cos[i][3] for i in components]
ten_perc_prec = [precision_dot_corr_cos[i][4] for i in components]
thir_perc_prec = [precision_dot_corr_cos[i][5] for i in components]

one_perc_prec.insert(0,'1%')
two_perc_prec.insert(0,'5%')
four_perc_prec.insert(0,'7%')
six_perc_prec.insert(0,'9%')
ten_perc_prec.insert(0,'10%')
thir_perc_prec.insert(0,'20%')


one_perc.insert(0,'1%')
two_perc.insert(0,'5%')
four_perc.insert(0,'7%')
six_perc.insert(0,'9%')
ten_perc.insert(0,'10%')
thirty_perc.insert(0,'20%')


######
one_perc_f1 = [f1_score_dot_corr_cos[i][0] for i in components]
two_perc_f1 = [f1_score_dot_corr_cos[i][1] for i in components]
four_perc_f1 = [f1_score_dot_corr_cos[i][2] for i in components]
six_perc_f1 = [f1_score_dot_corr_cos[i][3] for i in components]
ten_perc_f1 = [f1_score_dot_corr_cos[i][4] for i in components]
thir_perc_f1 = [f1_score_dot_corr_cos[i][5] for i in components]

one_perc_f1.insert(0,'1%')
two_perc_f1.insert(0,'5%')
four_perc_f1.insert(0,'7%')
six_perc_f1.insert(0,'9%')
ten_perc_f1.insert(0,'10%')
thir_perc_f1.insert(0,'20%')


one_perc_acc = [accuracy_dot_corr_cos[i][0] for i in components]
two_perc_acc = [accuracy_dot_corr_cos[i][1] for i in components]
four_perc_acc = [accuracy_dot_corr_cos[i][2] for i in components]
six_perc_acc = [accuracy_dot_corr_cos[i][3] for i in components]
ten_perc_acc = [accuracy_dot_corr_cos[i][4] for i in components]
thir_perc_acc = [accuracy_dot_corr_cos[i][5] for i in components]



one_perc_acc.insert(0,'1%')
two_perc_acc.insert(0,'5%')
four_perc_acc.insert(0,'7%')
six_perc_acc.insert(0,'9%')
ten_perc_acc.insert(0,'10%')
thir_perc_acc.insert(0,'20%')



print('Detection Rate')
print(tabulate([one_perc, two_perc, four_perc, six_perc, ten_perc, thirty_perc], headers=headers))
print("\n \n ")
print('Precision')
print(tabulate([one_perc_prec, two_perc_prec, four_perc_prec, six_perc_prec, ten_perc_prec, thir_perc_prec], headers=headers))
print("\n \n ")
print('F1_Score')
print(tabulate([one_perc_f1, two_perc_f1, four_perc_f1, six_perc_f1, ten_perc_f1, thir_perc_f1], headers=headers))
print("\n \n ")
print('Accuracy')
print(tabulate([one_perc_acc, two_perc_acc, four_perc_acc, six_perc_acc, ten_perc_acc, thir_perc_acc], headers=headers))


Detection Rate
FAR       PCA30     PCA40     PCA50     PCA60     PCA70
-----  --------  --------  --------  --------  --------
1%     0.943263  0.985759  0.985381  0.986767  0.986263
5%     0.961612  0.987044  0.986566  0.987901  0.986767
7%     0.989262  0.988027  0.987498  0.989086  0.988103
9%     0.99022   0.989439  0.987851  0.989641  0.988481
10%    0.990775  0.990397  0.989918  0.989993  0.989363
20%    0.991556  0.991808  0.991077  0.991985  0.995992

 
 
Precision
FAR       PCA30     PCA40     PCA50     PCA60     PCA70
-----  --------  --------  --------  --------  --------
1%     0.950908  0.953878  0.955867  0.967215  0.959373
5%     0.91729   0.93052   0.92859   0.941371  0.935885
7%     0.854983  0.86511   0.871145  0.887063  0.872183
9%     0.805437  0.811792  0.826487  0.840426  0.826491
10%    0.7623    0.75905   0.787433  0.813121  0.783723
20%    0.727543  0.716439  0.75011   0.78425   0.748674

 
 
F1_Score
FAR       PCA30     PCA40     PCA50     PCA60     PCA70
----

In [17]:
dictionary_major = {}
dictionary_minor = {}

dictionary_major_corr = {}
dictionary_minor_corr = {}

dictionary_major_cosine = {}
dictionary_minor_cosine = {}

for e,pca in enumerate(PCAs):
    
    r = len(pca.explained_variance_[pca.explained_variance_ < 0.20])
    
    p = len(pca.explained_variance_)
    

    out_threshold_list_major = []
    out_threshold_list_minor = []
    
    out_threshold_list_major_cosine = []
    out_threshold_list_minor_cosine = []
    
    out_threshold_list_major_corr = []
    out_threshold_list_minor_corr = []
    
    emp_distribution_major = []
    emp_distribution_minor = []
    
    emp_distribution_major_corr = []
    emp_distribution_minor_corr = []
    
    emp_distribution_major_cosine = []
    emp_distribution_minor_cosine = []
    
    for j in range(len(trains[e])):
        
        sample = np.array(trains[e].iloc[j])
        
        y = np.dot(sample,pca.components_.T)
        y_corr = np.corrcoef(sample,pca.components_)[0][1:]
        
        y_cosine = cosine_similarity(sample.reshape(1,-1),pca.components_)[0]
                
        s_major = np.sum(y[:pca.components_retained_]**2 / pca.explained_variance_[:pca.components_retained_])
        
        s_major_corr = np.sum(y_corr[:pca.components_retained_]**2 / pca.explained_variance_[:pca.components_retained_])
        
        s_major_cosine = np.sum(y_cosine[:pca.components_retained_]**2 / pca.explained_variance_[:pca.components_retained_])
        
        if len((np.where(np.isclose(pca.explained_variance_,0)))[0])!=0:
            s_minor = np.sum(y[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]]**2 / pca.explained_variance_[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]])
            s_minor_corr = np.sum(y_corr[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]]**2 / pca.explained_variance_[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]])
            s_minor_cosine = np.sum(y_cosine[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]]**2 / pca.explained_variance_[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]])
        else:
            s_minor = np.sum(y[p-r:]**2 / pca.explained_variance_[p-r:])
            s_minor_corr = np.sum(y_corr[p-r:]**2 / pca.explained_variance_[p-r:])
            s_minor_cosine = np.sum(y_cosine[p-r:]**2 / pca.explained_variance_[p-r:])
            
        emp_distribution_major.append(s_major)
        emp_distribution_minor.append(s_minor)
        
        emp_distribution_major_corr.append(s_major_corr)
        emp_distribution_minor_corr.append(s_minor_corr)
        
        emp_distribution_major_cosine.append(s_major_cosine)
        emp_distribution_minor_cosine.append(s_minor_cosine)

    for _, q in enumerate(quantils):
        
        
        n_major = len(emp_distribution_major)
        n_minor = len(emp_distribution_minor)

        n_major_corr = len(emp_distribution_major_corr)
        n_minor_corr = len(emp_distribution_minor_corr)
        
        n_major_cosine = len(emp_distribution_major_cosine)
        n_minor_cosine = len(emp_distribution_minor_cosine)
        
        sort_major = sorted(emp_distribution_major)
        sort_minor = sorted(emp_distribution_minor)
        
        sort_major_corr = sorted(emp_distribution_major_corr)
        sort_minor_corr = sorted(emp_distribution_minor_corr)
        
        sort_major_cosine = sorted(emp_distribution_major_cosine)
        sort_minor_cosine = sorted(emp_distribution_minor_cosine)
        
        out_threshold_major = sort_major[int(n_major * q)]
        out_threshold_minor = sort_minor[int(n_minor * q)]
        
        out_threshold_major_corr = sort_major_corr[int(n_major_corr * q)]
        out_threshold_minor_corr = sort_minor_corr[int(n_minor_corr * q)]
        
        out_threshold_major_cosine = sort_major_cosine[int(n_major_cosine * q)]
        out_threshold_minor_cosine = sort_minor_cosine[int(n_minor_cosine * q)]
        
        
        out_threshold_list_major.append(out_threshold_major)
        out_threshold_list_minor.append(out_threshold_minor)
        
        out_threshold_list_major_corr.append(out_threshold_major_corr)
        out_threshold_list_minor_corr.append(out_threshold_minor_corr)
        
        out_threshold_list_major_cosine.append(out_threshold_major_cosine)
        out_threshold_list_minor_cosine.append(out_threshold_minor_cosine)
        

    dictionary_major.update({pca.name : out_threshold_list_major})
    dictionary_minor.update({pca.name : out_threshold_list_minor})
    
    dictionary_major_corr.update({pca.name:out_threshold_list_major_corr})
    dictionary_minor_corr.update({pca.name:out_threshold_list_minor_corr})
    
    dictionary_major_cosine.update({pca.name:out_threshold_list_major_cosine})
    dictionary_minor_cosine.update({pca.name:out_threshold_list_minor_cosine})


In [36]:
# First experiment
components = [i for i in classification_res]
if experiment == 0:

    headers = ['FAR','PCA30','PCA40','PCA50','PCA60','PCA70']
    one_perc = [classification_res[i][0] for i in components]
    two_perc = [classification_res[i][1] for i in components]
    four_perc = [classification_res[i][2] for i in components]
    six_perc = [classification_res[i][3] for i in components]
    eight_perc = [classification_res[i][4] for i in components]
    ten_perc = [classification_res[i][5] for i in components]


    one_perc.insert(0,'1%')
    two_perc.insert(0,'2%')
    four_perc.insert(0,'4%')
    six_perc.insert(0,'6%')
    eight_perc.insert(0,'8%')
    ten_perc.insert(0,'10%')

    print(tabulate([one_perc, two_perc, four_perc, six_perc, eight_perc, ten_perc], headers=headers))
else:
    #Second experiment
    headers = ['FAR','PCC']
    one_perc = [classification_res[i][0] for i in components]
    two_perc = [classification_res[i][1] for i in components]
    four_perc = [classification_res[i][2] for i in components]
    six_perc = [classification_res[i][3] for i in components]
    eight_perc = [classification_res[i][4] for i in components]
    ten_perc = [classification_res[i][5] for i in components]

    one_perc_avg = np.mean(one_perc)
    one_perc_std = np.std(one_perc)

    two_perc_avg = np.mean(two_perc)
    two_perc_std = np.std(two_perc)

    four_perc_avg = np.mean(four_perc)
    four_perc_std = np.std(four_perc)

    six_perc_avg = np.mean(six_perc)
    six_perc_std = np.std(six_perc)


    eight_perc_avg = np.mean(eight_perc)
    eight_perc_std = np.std(eight_perc)

    ten_perc_avg = np.mean(ten_perc)
    ten_perc_std = np.std(ten_perc)

    one_str = [str(one_perc_avg)+' +/- '+str(one_perc_std)]
    two_str = [str(two_perc_avg)+' +/- '+str(two_perc_std)]
    four_str = [str(four_perc_avg)+' +/- '+str(four_perc_std)]
    six_str = [str(six_perc_avg)+' +/- '+str(six_perc_std)]
    eight_str =[ str(eight_perc_avg)+' +/- '+str(eight_perc_std)]
    ten_str = [str(ten_perc_avg)+' +/- '+str(ten_perc_std)]

    one_str.insert(0,'1%')
    two_str.insert(0,'2%')
    four_str.insert(0,'4%')
    six_str.insert(0,'6%')
    eight_str.insert(0,'8%')
    ten_str.insert(0,'10%')

    print(tabulate([one_str, two_str, four_str, six_str, eight_str, ten_str], headers=headers))


FAR       PCA30     PCA40     PCA50     PCA60     PCA70
-----  --------  --------  --------  --------  --------
1%     0.671321  0.957632  0.932864  0.935224  0.931282
2%     0.688184  0.985287  0.972117  0.961742  0.931737
4%     0.711751  0.985692  0.984647  0.986083  0.960357
6%     0.713616  0.986779  0.985907  0.987298  0.985058
8%     0.713893  0.987259  0.987218  0.988006  0.986727
10%    0.714196  0.987436  0.987445  0.988993  0.994387


In [71]:
components = [i for i in classification_res]
headers = ['FAR','PCC']
one_perc = [classification_res[i][0] for i in components]
two_perc = [classification_res[i][1] for i in components]
four_perc = [classification_res[i][2] for i in components]
six_perc = [classification_res[i][3] for i in components]
eight_perc = [classification_res[i][4] for i in components]
ten_perc = [classification_res[i][5] for i in components]

one_perc_avg = np.mean(one_perc)
one_perc_std = np.std(one_perc)

two_perc_avg = np.mean(two_perc)
two_perc_std = np.std(two_perc)

four_perc_avg = np.mean(four_perc)
four_perc_std = np.std(four_perc)

six_perc_avg = np.mean(six_perc)
six_perc_std = np.std(six_perc)


eight_perc_avg = np.mean(eight_perc)
eight_perc_std = np.std(eight_perc)

ten_perc_avg = np.mean(ten_perc)
ten_perc_std = np.std(ten_perc)

one_str = [str(one_perc_avg)+' +/- '+str(one_perc_std)]
two_str = [str(two_perc_avg)+' +/- '+str(two_perc_std)]
four_str = [str(four_perc_avg)+' +/- '+str(four_perc_std)]
six_str = [str(six_perc_avg)+' +/- '+str(six_perc_std)]
eight_str =[ str(eight_perc_avg)+' +/- '+str(eight_perc_std)]
ten_str = [str(ten_perc_avg)+' +/- '+str(ten_perc_std)]

one_str.insert(0,'1%')
two_str.insert(0,'2%')
four_str.insert(0,'4%')
six_str.insert(0,'6%')
eight_str.insert(0,'8%')
ten_str.insert(0,'10%')

print(tabulate([one_str, two_str, four_str, six_str, eight_str, ten_str], headers=headers))


FAR    PCC
-----  --------------------------------------------
1%     0.9773046382009991 +/- 0.017277049377979803
2%     0.9817819997535467 +/- 0.01035542644800812
4%     0.9882759318536403 +/- 0.0006692154240590303
6%     0.9890678895858972 +/- 0.0008632678300970236
8%     0.9900158672421652 +/- 0.0004825865447686921
10%    0.9924270670645295 +/- 0.0017969092422338239


# Quantum

# ---------------------------------------------------------------------------------------------------------------
## No tomography

In [7]:
'''qpca30 = qPCA(svd_solver='full').fit(trains[0].drop(columns='41'),p=0.30,estimate_all=True,delta=0.1,eps=0.2,theta=0.901703)
qpca40 = qPCA(svd_solver='full').fit(trains[1].drop(columns='41'),p=0.40,estimate_all=True,delta=0.1,eps=0.2,theta =0.624750)
qpca50 = qPCA(svd_solver='full').fit(trains[2].drop(columns='41'),p=0.50,estimate_all=True,delta=0.1,eps=0.2,theta = 0.575973)
qpca60 = qPCA(svd_solver='full').fit(trains[3].drop(columns='41'),p=0.60,estimate_all=True,delta=0.1,eps=0.2,theta =0.475055)
qpca70 = qPCA(svd_solver='full').fit(trains[4].drop(columns='41'),p=0.70,estimate_all=True,delta=0.1,eps=0.2,theta =0.474014)
'''
qpca30 = qPCA(svd_solver='full').fit(trains[0].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=0.30,estimate_all=True,delta=0.1,eps=0.2)
qpca40 = qPCA(svd_solver='full').fit(trains[1].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=0.40,estimate_all=True,delta=0.1,eps=0.2)
qpca50 = qPCA(svd_solver='full').fit(trains[2].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=0.50,estimate_all=True,delta=0.1,eps=0.2)
qpca60 = qPCA(svd_solver='full').fit(trains[3].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=0.60,estimate_all=True,delta=0.1,eps=0.2)
qpca70 = qPCA(svd_solver='full').fit(trains[4].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=0.70,estimate_all=True,delta=0.1,eps=0.2)
QPCAs = [qpca30,qpca40,qpca50,qpca60,qpca70]


Theta: [0.90247859] Error: [-0.00135207]
noise_per_comp: 0.012126781251816649 errors: [-0.0097487   0.00079794 -0.0114998   0.00928063  0.0108233  -0.0039646
  0.00607312 -0.00493892  0.00571551 -0.00952257 -0.00369635 -0.00200026
  0.00487826  0.00505328  0.003159   -0.00256295 -0.00645512  0.00482693
  0.01106496 -0.01079774 -0.01154397 -0.00505981  0.0090378  -0.00799247
 -0.01057259 -0.00037232 -0.0089282  -0.00659948  0.00024789  0.00679228
  0.00865671 -0.00308913  0.0085152   0.00102137  0.00653822 -0.01001813
  0.01055523 -0.00423117  0.00453059 -0.00493396 -0.00821058 -0.00814421
  0.00358479  0.00212556 -0.00688359  0.00908562 -0.00048196 -0.00612029
  0.00623767  0.00646211  0.00135767  0.00465548  0.0079256  -0.00693078
 -0.00045902 -0.00866604  0.00094628  0.01012286 -0.00561428  0.00501957
 -0.01090642 -0.00459269 -0.00655453 -0.00476134  0.00131064  0.00532034
 -0.00264531  0.00291712]
noise_per_comp: 0.012126781251816649 errors: [-0.00591761 -0.00563158 -0.00770506  0.0

# ---------------------------------------------------------------------------------------------------------------
## Tomography

In [27]:
qpca30 = qPCA(svd_solver='full').fit(trains[0],theta_estimate=True,eps_theta=0.005,p=0.30,estimate_all=True,delta=0.5,eps=0.4,fake_tomography=False)
qpca40 = qPCA(svd_solver='full').fit(trains[1],theta_estimate=True,eps_theta=0.005,p=0.40,estimate_all=True,delta=0.5,eps=.4,fake_tomography=False)
qpca50 = qPCA(svd_solver='full').fit(trains[2],theta_estimate=True,eps_theta=0.005,p=0.50,estimate_all=True,delta=0.5,eps=0.4,fake_tomography=False)
qpca60 = qPCA(svd_solver='full').fit(trains[3],theta_estimate=True,eps_theta=0.005,p=0.60,estimate_all=True,delta=0.5,eps=0.4,fake_tomography=False)
qpca70 = qPCA(svd_solver='full').fit(trains[4],theta_estimate=True,eps_theta=0.005,p=0.70,estimate_all=True,delta=0.5,eps=0.4,fake_tomography=False)
QPCAs = [qpca30,qpca40,qpca50,qpca60,qpca70]

2
Theta:  [0.90121116] Error:  [-0.0026195]
noise_per_comp: 0.282842712474619 errors: [-0.21042341  0.28241087]
noise_per_comp: 0.565685424949238 errors: [0.03253736 0.20279299]
noise_per_comp: 0.06859943405700354 errors: [ 0.04417464 -0.00594825  0.03308074  0.01940125 -0.03058048  0.0463312
 -0.04683107  0.0142555  -0.00690613  0.03252576 -0.0374877  -0.04564672
 -0.02646529 -0.00852432 -0.01451944  0.01418988  0.06280188  0.02466151
  0.060264   -0.00499667 -0.06843778  0.01355405 -0.00948847  0.02192737
  0.03864162 -0.05886639 -0.04072433  0.00781819  0.03953915 -0.02436491
 -0.01155535  0.03948624 -0.06284485 -0.03331516]
4
Theta:  [0.62098923] Error:  [-0.00062295]
noise_per_comp: 0.2 errors: [-0.0949598   0.15446633 -0.1890082  -0.17888759]
noise_per_comp: 0.4 errors: [-0.03344394 -0.32612736 -0.14115206  0.18432786]
noise_per_comp: 0.06859943405700354 errors: [-0.06762961 -0.06149831  0.05125697 -0.04307427  0.02417706 -0.00903276
 -0.02126638 -0.06291064 -0.05222324 -0.059556

In [28]:
qpca30.estimate_fs

array([4.70478674, 4.0195996 ])

In [29]:
dictionary_major = {}
dictionary_minor  = {}
for e,pca in enumerate(QPCAs):


    out_threshold_list_major = []
    out_threshold_list_minor = []
    emp_distribution_major = []
    
    
    for j in range(len(trains[e])):
        sample = np.array(trains[e].iloc[j])
        y = np.dot(sample,pca.estimate_right_sv.T)
        s_major = np.sum(y**2 / pca.estimate_fs)

        emp_distribution_major.append(s_major)
        

    for q in quantils:
        n_major = len(emp_distribution_major)
        
        sort_major = sorted(emp_distribution_major)
        
        out_threshold_major = sort_major[int(n_major * q)]
        
        out_threshold_list_major.append(out_threshold_major)
        
    
    dictionary_major.update({pca:out_threshold_list_major})

In [17]:
dictionary_major

{qPCA(svd_solver='full'): [24.11249146526805,
  22.168919390631043,
  19.34527839153628,
  18.010394126163135,
  16.912877440427227,
  15.91276234017994],
 qPCA(svd_solver='full'): [32.97833286547563,
  24.635780335232393,
  20.69711710195754,
  19.334202360246202,
  18.016380414112316,
  16.79712851320233],
 qPCA(svd_solver='full'): [97.01457576185626,
  34.30036040466233,
  21.69788242079512,
  18.794273427978027,
  17.32327895235951,
  16.297143699340914],
 qPCA(svd_solver='full'): [129.26041485247217,
  64.93431646227016,
  44.282981856050164,
  37.98419787834,
  32.073974284543894,
  25.885465240869593],
 qPCA(svd_solver='full'): [229.00555703501084,
  132.52596860997164,
  68.14841535597228,
  51.41577047776559,
  38.04582900549914,
  29.776217720199508]}

In [30]:
classification_res = {}
for e,pca in enumerate(QPCAs):
    
    acc_list  = []
    for threshold_major in dictionary_major[pca]:
        print(threshold_major)
        TP = 0
        FP = 0
        TN = 0
        FN = 0
        dotted = tests[e].dot(pca.estimate_right_sv.T)
        sum_major = np.sum(dotted**2 / pca.estimate_fs,axis = 1)
        attack_prediction = test[e].iloc[np.where(sum_major > threshold_major)[0]][41].value_counts()
        normal_prediction = test[e].iloc[np.where(sum_major <= threshold_major)[0]][41].value_counts()
        total_attack = attack_prediction.sum()
        
        TP = attack_prediction['attack']
        FP = total_attack-TP
        
        total_normal = normal_prediction.sum()
        TN = normal_prediction['normal.']
        FN = total_normal-TN

        recall = (TP)/(TP+FN)
        precision = (TP)/(TP+FP)
        print(recall, precision)
        acc_list.append(recall)
        

    classification_res.update({pca:acc_list})

20.85467896309747
0.6590714321722034 0.9848958529511469
19.07780385502391
0.6639864898926249 0.9702405067953298
17.564641960432027
0.6787568684780965 0.9400942572874847
16.543120670083834
0.6943842314866159 0.9143378692333223
15.461231045154554
0.7086252961637345 0.8869329295223674
14.480082611274355
0.710591319251903 0.8646791804686541
69.11786040223255
0.9540757170943187 0.9847290512240173
32.75548876004544
0.9816504511770933 0.974575847054702
23.66584718772406
0.9827342844180068 0.9575607240219073
20.097330964559834
0.983969350204164 0.936994455512085
18.300500705769416
0.984322226143066 0.9193032015065913
16.929092797393977
0.9845238695367243 0.896817743490839
116.4276676251938
0.9300801532489792 0.9935112140222395
68.26411898360675
0.9553359883046831 0.9821715470329101
55.40170205671008
0.9746685486716742 0.960171827278822
32.408076808273414
0.9829107223874578 0.9349540866479656
22.5803144623467
0.9842970207188587 0.911618460676518
19.583277528390838
0.9845994858093462 0.895960916

In [19]:
components = [i for i in classification_res]


In [20]:
#Tomography delta = 0.1, eps = 0.2

headers = ['FAR','PCA30','PCA40','PCA50','PCA60','PCA70']
one_perc = [classification_res[i][0] for i in components]
two_perc = [classification_res[i][1] for i in components]
four_perc = [classification_res[i][2] for i in components]
six_perc = [classification_res[i][3] for i in components]
eight_perc = [classification_res[i][4] for i in components]
ten_perc = [classification_res[i][5] for i in components]


one_perc.insert(0,'1%')
two_perc.insert(0,'2%')
four_perc.insert(0,'4%')
six_perc.insert(0,'6%')
eight_perc.insert(0,'8%')
ten_perc.insert(0,'10%')

print(tabulate([one_perc, two_perc, four_perc, six_perc, eight_perc, ten_perc], headers=headers))

FAR       PCA30     PCA40     PCA50     PCA60     PCA70
-----  --------  --------  --------  --------  --------
1%     0.661163  0.930887  0.929626  0.93066   0.928467
2%     0.675455  0.958109  0.972425  0.9571    0.931618
4%     0.702727  0.981524  0.982886  0.983112  0.976937
6%     0.710012  0.982659  0.983667  0.983919  0.98349
8%     0.710264  0.983213  0.984146  0.984347  0.984297
10%    0.710743  0.98344   0.984247  0.985431  0.990195


# 2nd Experiment Quantum

In [7]:
qpca30 = qPCA(svd_solver='full',name = 'pca30').fit(trains[0],theta_estimate=True,eps_theta=0.005,p=1,estimate_all=True,delta=0.5,eps=0.4,fake_tomography=False)
qpca40 = qPCA(svd_solver='full',name = 'pca40').fit(trains[1],theta_estimate=True,eps_theta=0.005,p=1,estimate_all=True,delta=0.5,eps=0.4,fake_tomography=False)
qpca50 = qPCA(svd_solver='full',name = 'pca50').fit(trains[2],theta_estimate=True,eps_theta=0.005,p=1,estimate_all=True,delta=0.5,eps=0.4,fake_tomography=False)
qpca60 = qPCA(svd_solver='full',name = 'pca60').fit(trains[3],theta_estimate=True,eps_theta=0.005,p=1,estimate_all=True,delta=0.5,eps=0.4,fake_tomography=False)
qpca70 = qPCA(svd_solver='full',name = 'pca70').fit(trains[4],theta_estimate=True,eps_theta=0.005,p=1,estimate_all=True,delta=0.5,eps=0.4,fake_tomography=False)
QPCAs=[qpca30,qpca40,qpca50,qpca60,qpca70]

35
Theta:  [0.00089091] Error:  [0.00089091]
noise_per_comp: 0.0769800358919501 errors: [ 7.11220632e-02  5.15338282e-02  3.72665559e-02  3.87158468e-02
  1.76667842e-02  6.80844005e-02  3.72432664e-05 -6.61677479e-02
  2.18869257e-02 -1.03386562e-02  9.39367601e-03 -6.48542811e-02
  6.41745421e-02 -2.89455495e-02  5.16469319e-02  6.33388457e-02
  9.24830452e-03 -1.05334894e-02  9.24427442e-04  5.39139844e-02
  3.37299432e-02 -3.01816171e-02 -6.71828922e-03  5.53982172e-02
  4.43916152e-02 -4.73467690e-02 -8.69599693e-04]
noise_per_comp: 0.1539600717839002 errors: [ 0.033172   -0.08190934  0.08412355 -0.05924485  0.03651275  0.03652543
 -0.08839677  0.09694315 -0.12976508  0.08751628 -0.10731467  0.09103179
 -0.02218699  0.08847032  0.03473876 -0.10691399 -0.10425092  0.11312142
 -0.13048302 -0.14562885 -0.00339192  0.01288388 -0.08045777 -0.01773585
  0.02684114 -0.0715569  -0.06708106]
noise_per_comp: 0.06859943405700354 errors: [-4.85345476e-02  8.04357178e-03  3.20095470e-02 -6.347

In [8]:
qmodel =  Model(QPCAs, quantils,quantum = True).fit(trains,minor_sv_variance=0.20)

recall, precision, accuracy, f1_score = qmodel.predict(tests, labels, name_negative_class='normal.', only_dot_product=True, experiment=1)


exp1
pca30
21.451904087796237 209.08222327637128
detection_rate: 0.9829107223874578
precision: 0.9792576967505399
accuracy: 0.9886019158481872
F1_score: 0.9810808090973131
TP: 38996 TN: 91452 FP: 826 FN: 678 TOT_SAMPLES: 131952 LEN_TEST: 131952
19.519289256589943 109.01277210561597
detection_rate: 0.9844230478398951
precision: 0.9614731296619975
accuracy: 0.9834561052503941
F1_score: 0.9728127529734105
TP: 39056 TN: 90713 FP: 1565 FN: 618 TOT_SAMPLES: 131952 LEN_TEST: 131952
17.446997991851735 72.14378962057454
detection_rate: 0.9865907143217221
precision: 0.9204900877172354
accuracy: 0.9703452770704498
F1_score: 0.9523948562599609
TP: 39142 TN: 88897 FP: 3381 FN: 532 TOT_SAMPLES: 131952 LEN_TEST: 131952
16.32643208405831 62.92726569038779
detection_rate: 0.988329888592025
precision: 0.8863847005900038
accuracy: 0.9584015399539226
F1_score: 0.9345854536353995
TP: 39211 TN: 87252 FP: 5026 FN: 463 TOT_SAMPLES: 131952 LEN_TEST: 131952
15.554873413307606 52.544936996511495
detection_rate: 

In [24]:
dictionary_major = {}
dictionary_minor  = {}

ret_variance = [0.30,0.40,0.50,0.60,0.70]

for e,pca in enumerate(QPCAs):
    
    r = len(pca.estimate_fs[pca.estimate_fs < 0.20])
    
    p = len(pca.estimate_fs)

    out_threshold_list_major = []
    out_threshold_list_minor = []
    emp_distribution_major = []
    emp_distribution_minor = []
    
    for j in range(len(trains[e])):
        sample = np.array(trains[e].iloc[j])
        y = np.dot(sample,pca.estimate_right_sv.T)
        s_major = np.sum(y[:np.searchsorted(stable_cumsum(pca.estimate_fs_ratio), ret_variance[e], side='right') + 1]**2 /
                         pca.estimate_fs[:np.searchsorted(stable_cumsum(pca.estimate_fs_ratio), ret_variance[e], side='right') + 1])
        
        if any(np.isclose(pca.estimate_fs,0)):
            #Remove almost zero eigenvalues
            s_minor = np.sum(y[p-r:np.where(np.isclose(pca.estimate_fs,0))[0][0]]**2 / 
                             pca.estimate_fs[p-r:np.where(np.isclose(pca.estimate_fs,0))[0][0]])
            
        else:
            s_minor = np.sum(y[p-r:]**2 / 
                             pca.estimate_fs[p-r:]) 
        

        emp_distribution_major.append(s_major)
        emp_distribution_minor.append(s_minor)

    for q in quantils:
        n_major = len(emp_distribution_major)
        n_minor = len(emp_distribution_minor)

        sort_major = sorted(emp_distribution_major)
        sort_minor = sorted(emp_distribution_minor)
        out_threshold_major = sort_major[int(n_major * q)]
        out_threshold_minor = sort_minor[int(n_minor * q)]
        out_threshold_list_major.append(out_threshold_major)
        out_threshold_list_minor.append(out_threshold_minor)

    dictionary_major.update({pca:out_threshold_list_major})
    dictionary_minor.update({pca:out_threshold_list_minor})


In [25]:
classification_res = {}
for e,pca in enumerate(QPCAs):
    r = len(pca.estimate_fs[pca.estimate_fs < 0.20])
    p = len(pca.estimate_fs)

    acc_list  = []
    for threshold_major,threshold_minor in zip(dictionary_major[pca],dictionary_minor[pca]):
        print(threshold_major,threshold_minor)
        TP =[]
        FP = []
        TN = []
        FN = []

        for j in range(len(tests[e])):
            sample = np.array(tests[e].iloc[j])

            y = np.dot(sample, pca.estimate_right_sv.T)

            sum_major = np.sum(y[:np.searchsorted(stable_cumsum(pca.estimate_fs_ratio), ret_variance[e], side='right') + 1]**2 /
                             pca.estimate_fs[:np.searchsorted(stable_cumsum(pca.estimate_fs_ratio), ret_variance[e], side='right') + 1])

            if any(np.isclose(pca.estimate_fs,0)):
                sum_minor = np.sum(y[p-r:np.where(np.isclose(pca.estimate_fs,0))[0][0]]**2 / 
                                 pca.estimate_fs[p-r:np.where(np.isclose(pca.estimate_fs,0))[0][0]])

            else:
                sum_minor = np.sum(y[p-r:]**2 / 
                                 pca.estimate_fs[p-r:]) 


            if sum_major > threshold_major or sum_minor > threshold_minor:
                if test[e].iloc[j][41] == 'attack':
                    TP.append(j)
                else:
                    FP.append(j)
            else:
                if sum_major <= threshold_major and sum_minor <= threshold_minor:

                    if test[e].iloc[j][41] == 'attack':
                        FN.append(j)
                    else:
                        TN.append(j)

        accuracy = (len(TP))/(len(TP)+len(FN))
        print(accuracy)
        acc_list.append(accuracy)

    classification_res.update({pca:acc_list})

21.844347714477944 73.5201008143925
0.9844504563727845
19.751932779738578 42.69797464720304
0.9855376602361507
17.86144836247983 26.163485663151615
0.9871052565042603
16.67168563884435 15.870295667510879
0.9875350812874516
15.818619126530272 9.380690581301097
0.9885211499077141
14.722006747387102 6.571340795367283
0.9890268261232333
31.39680830141107 148.636030464715
0.982834028567817
22.608621105387293 134.98389420367891
0.9836177474402731
18.56257598325613 24.87554456373263
0.9855896852483883
17.237411601424906 11.413574036434214
0.9862722791050436
16.057017724244286 7.86036045555724
0.986575654152446
15.125526681217936 6.8496829110613975
0.9868790291998483
118.61811740903116 75.0801880078677
0.9377132890113501
58.534842579700864 53.78722521350422
0.9742662858009555
47.803661537244665 30.38772586250744
0.9885993073636846
29.310196457776684 23.460941374858187
0.9902171440127404
22.707198872054622 15.88265641457198
0.9913799641042493
19.525939556600882 12.013613290757727
0.992492226800

In [9]:
components = [i for i in recall]
headers = ['FAR','PCC']
one_perc = [recall[i][0] for i in components]
two_perc = [recall[i][1] for i in components]
four_perc = [recall[i][2] for i in components]
six_perc = [recall[i][3] for i in components]
eight_perc = [recall[i][4] for i in components]
ten_perc = [recall[i][5] for i in components]

one_perc_avg = np.mean(one_perc)
one_perc_std = np.std(one_perc)

two_perc_avg = np.mean(two_perc)
two_perc_std = np.std(two_perc)

four_perc_avg = np.mean(four_perc)
four_perc_std = np.std(four_perc)

six_perc_avg = np.mean(six_perc)
six_perc_std = np.std(six_perc)


eight_perc_avg = np.mean(eight_perc)
eight_perc_std = np.std(eight_perc)

ten_perc_avg = np.mean(ten_perc)
ten_perc_std = np.std(ten_perc)

one_str = [str(one_perc_avg)+' +/- '+str(one_perc_std)]
two_str = [str(two_perc_avg)+' +/- '+str(two_perc_std)]
four_str = [str(four_perc_avg)+' +/- '+str(four_perc_std)]
six_str = [str(six_perc_avg)+' +/- '+str(six_perc_std)]
eight_str =[ str(eight_perc_avg)+' +/- '+str(eight_perc_std)]
ten_str = [str(ten_perc_avg)+' +/- '+str(ten_perc_std)]

one_str.insert(0,'1%')
two_str.insert(0,'2%')
four_str.insert(0,'4%')
six_str.insert(0,'6%')
eight_str.insert(0,'8%')
ten_str.insert(0,'10%')

print(tabulate([one_str, two_str, four_str, six_str, eight_str, ten_str], headers=headers))

FAR    PCC
-----  --------------------------------------------
1%     0.9782628421636337 +/- 0.009220702288085638
2%     0.9836618440288349 +/- 0.0004632315373356831
4%     0.9847557594394314 +/- 0.000988798222407924
6%     0.9858900035287593 +/- 0.0015075625288854429
8%     0.986852850733478 +/- 0.001427134590924542
10%    0.9890658869788778 +/- 0.0017956030019246746


# Find best parameters

In [14]:
qpca30 = qPCA(svd_solver='full',name = 'PCA30').fit(trains[0],theta_estimate=True,eps_theta=0.05,p=0.30,estimate_all=True,delta=0.5,eps=0.4,tomography=True)
qpca40 = qPCA(svd_solver='full',name = 'PCA40').fit(trains[1],theta_estimate=True,eps_theta=0.05,p=0.40,estimate_all=True,delta=0.5,eps=0.4,tomography=True)
qpca50 = qPCA(svd_solver='full', name = 'PCA50').fit(trains[2],theta_estimate=True,eps_theta=0.05,p=0.50,estimate_all=True,delta=0.5,eps=0.4,tomography=True)
qpca60 = qPCA(svd_solver='full', name = 'PCA60').fit(trains[3],theta_estimate=True,eps_theta=0.05,p=0.60,estimate_all=True,delta=0.5,eps=0.4,tomography=True)
qpca70 = qPCA(svd_solver='full',name = 'PCA70').fit(trains[4],theta_estimate=True,eps_theta=0.05,p=0.70,estimate_all=True,delta=0.5,eps=0.4,tomography=True)

QPCAs = [qpca30,qpca40,qpca50,qpca60,qpca70]

Theta:  [0.87813891] Error:  [-0.02569175]
noise_per_comp: 0.282842712474619 errors: [0.04678794 0.04225981]
noise_per_comp: 0.06859943405700354 errors: [ 0.0079547  -0.02468898 -0.05532919 -0.05711353 -0.06728477 -0.05470389
 -0.03079133  0.0403923  -0.06214322 -0.06106835 -0.05503233  0.06560048
 -0.05902177  0.03250007  0.02965686  0.0342265  -0.00498663 -0.0149293
  0.02872104  0.04026383  0.06291662 -0.06206277 -0.05620818 -0.00733844
 -0.02006731 -0.05902073 -0.06375772 -0.02620712  0.06337163 -0.03838271
  0.04016952  0.03549021  0.03505709  0.04626028]
Theta:  [0.63220486] Error:  [0.01059267]
noise_per_comp: 0.23094010767585033 errors: [-0.15084357 -0.05314009 -0.04879821]
noise_per_comp: 0.06859943405700354 errors: [-0.05190702 -0.06445144 -0.0673815  -0.0385806   0.06452846  0.01504951
  0.01569068  0.03597478 -0.05812872 -0.02967847 -0.01380108 -0.06751815
  0.05954761 -0.03703337  0.02528232  0.02477394  0.05732352 -0.03454757
 -0.03302362 -0.00386216 -0.00943376 -0.029519

 Numerical methods for large eigenvalues problems.

In [12]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt
from scipy.io import savemat
from numpy import sin,sqrt

n,m=np.meshgrid(np.linspace(0,100000,dtype=np.int64,num=100),np.linspace(0,500,dtype=np.int64,num=100))

n_dict={'n':n}
m_dict={'m':m}
path = os.path.join(cwd,'MATLABfile/KDDCUP')
savemat(os.path.join(path , 'n.mat'),n_dict)
savemat(os.path.join(path , 'm.mat'),m_dict)
##
def y_classic_rand(n,m,k,eps):
    #Faster Matrix Completion Using Randomized SVD svds
    return n * m * k * np.log(m / eps) / np.sqrt(eps)

def y_classic(n,m):
    return n * m**2

def y_quantum(cost, n, m, theta_estimation_cost, add_cost):
    return cost * m + theta_estimation_cost + add_cost

def additional_quantum_cost(n, m, A, epsilon, spectral_norm,train):
    return ( np.log2(1/epsilon)*np.linalg.norm(train)*np.log2(n*m)/(epsilon*spectral_norm))

##

def y_classic_rand(n,m,k,eps):
    #Faster Matrix Completion Using Randomized SVD svds
    return n * m * k * np.log(m / eps) / np.sqrt(eps)
def y_classic(n,m):
    return n * m**2

#def y_quantum(cost, n, m, add_cost):
#   return cost * n * np.log2(n) * np.log2(n * m) + add_cost
#SVD0 -> FULLSVD
#SVD1 -> RANDSVD
SVD = 0
theta_estimate_additional_cost = 1
additional_cost = 0

for index, qpca in enumerate(QPCAs):
    a = trains[index] / qpca.spectral_norm
    max_ = 0
    for i in range(len(a)):
        l1_norm = np.sum([abs(a_) for a_ in a.loc[i]])
        if l1_norm > max_:
            max_ = l1_norm
    
    NE = PCAs[index].explained_variance_ 
    
    k = PCAs[index].components_retained_

    p = qpca.p
    one_over_theta = 1 / qpca.est_theta
    one_over_sqrtp = 1 / np.sqrt(p)
    muA_over_eps = max_ / qpca.eps
    k_over_delta2 = len(qpca.estimate_fs) / (qpca.delta) ** 2
    
    cost = one_over_theta * one_over_sqrtp * muA_over_eps * k_over_delta2
    add_cost = max_ / qpca.eps_theta
    mpl.rcParams['legend.fontsize'] = 10

    if theta_estimate_additional_cost:
        theta_estimate_cost = max_ / qpca.eps_theta
    else:
        theta_estimate_cost = 0

    if additional_cost:
        add_cost = additional_quantum_cost(n, m, trains[index], qpca.eps, qpca.spectral_norm,trains[index])
    else:
        add_cost = 0
    z = y_quantum(cost, n, m, theta_estimate_cost, add_cost)
    if SVD ==1:
        z1 = y_classic_rand(n, m,PCAs[index].components_retained_,NE[k-1]-NE[k])
        classic_dict={'z1'+str(index):z1}
        quantum_dict={'z'+str(index):z}

        savemat(os.path.join(path,'RANDSVD','z_classic'+str(index)+'.mat'),classic_dict)
        savemat(os.path.join(path,'RANDSVD','z_quantum'+str(index)+'.mat'),quantum_dict)
    else:
        z1 = y_classic(n,m)
        classic_dict={'z1'+str(index):z1}
        quantum_dict={'z'+str(index):z}

        savemat(os.path.join(path,'FULLSVDExp1','z_classic'+str(index)+'.mat'),classic_dict)
        savemat(os.path.join(path,'FULLSVDExp1','z_quantum'+str(index)+'.mat'),quantum_dict)
    